In [74]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

import os
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Energy_Cancer").getOrCreate()

# Read in data
from pyspark import SparkFiles

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [127]:
import requests
url = "https://raw.githubusercontent.com/fraserclaire/Group3_Project4/main/Artien/Machine%20Learning%20Dataset-Cancer%26Energy.csv"
response = requests.get(url)
# with open("Machine Learning Dataset-Cancer&Energy.csv", "wb") as f:
#     f.write(response.content)

spark.sparkContext.addFile(url)
energy_cancer = spark.read.csv(SparkFiles.get('Machine Learning Dataset-Cancer&Energy.csv'),inferSchema=True,header=True)

# Convert Spark DataFrame to pandas DataFrame
energy_cancer_df = energy_cancer.toPandas()

# Show DataFrame
energy_cancer_df.head()

,_c0,State Abbreviation,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),...,2019_Population,2020_LungBronchus,2020_LungBronchus_perCapita,2020_Population,2021 Lung & Bronchus,2021_Population,2021_LungBronchus_perCapita,2011-2020 allsites/capita AVG,2011-2020 lungBronchus/capita AVG,Lung/Bronchus_Classifer
0,0,AK,15481,15521,14819,18225,19511,16613,16376,17325,...,731545,400,0.000546,732964.0,370,734923,0.000503,0.004628,0.000591,0
1,1,AL,651032,547004,565051,575912,494311,410168,378890,377152,...,4903185,4230,0.000841,5031864.0,4520,5050380,0.000895,0.005554,0.000867,1
2,2,AR,306119,296732,327099,339214,226889,246437,267591,304145,...,3017804,2760,0.000916,3014348.0,2970,3028443,0.000981,0.005473,0.000898,1
3,3,AZ,459909,420570,454865,447849,385822,323878,334512,331492,...,7278717,4200,0.000584,7186683.0,4550,7272487,0.000626,0.004939,0.000595,1
4,4,CA,55264,43832,38151,39486,30967,32077,33663,33256,...,39512223,18040,0.000457,39503200.0,17760,39145060,0.000454,0.004456,0.000473,0


In [128]:
energy_cancer_df.drop(columns='_c0', inplace=True)
energy_cancer_df.rename(columns={'Lung/Bronchus_Classifer': 'LungBronchus_Classifier'}, inplace=True)
energy_cancer_df.head()

,State Abbreviation,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),...,2019_Population,2020_LungBronchus,2020_LungBronchus_perCapita,2020_Population,2021 Lung & Bronchus,2021_Population,2021_LungBronchus_perCapita,2011-2020 allsites/capita AVG,2011-2020 lungBronchus/capita AVG,LungBronchus_Classifier
0,AK,15481,15521,14819,18225,19511,16613,16376,17325,17620,...,731545,400,0.000546,732964.0,370,734923,0.000503,0.004628,0.000591,0
1,AL,651032,547004,565051,575912,494311,410168,378890,377152,317180,...,4903185,4230,0.000841,5031864.0,4520,5050380,0.000895,0.005554,0.000867,1
2,AR,306119,296732,327099,339214,226889,246437,267591,304145,239798,...,3017804,2760,0.000916,3014348.0,2970,3028443,0.000981,0.005473,0.000898,1
3,AZ,459909,420570,454865,447849,385822,323878,334512,331492,257727,...,7278717,4200,0.000584,7186683.0,4550,7272487,0.000626,0.004939,0.000595,1
4,CA,55264,43832,38151,39486,30967,32077,33663,33256,30886,...,39512223,18040,0.000457,39503200.0,17760,39145060,0.000454,0.004456,0.000473,0


In [140]:
energy_cancer_wdummies = pd.get_dummies(energy_cancer_df, dtype='int')
energy_cancer_wdummies.head()

,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),2020(C),...,State_Tennessee,State_Texas,State_United States,State_Utah,State_Vermont,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,15481,15521,14819,18225,19511,16613,16376,17325,17620,18527,...,0,0,0,0,0,0,0,0,0,0
1,651032,547004,565051,575912,494311,410168,378890,377152,317180,256687,...,0,0,0,0,0,0,0,0,0,0
2,306119,296732,327099,339214,226889,246437,267591,304145,239798,161980,...,0,0,0,0,0,0,0,0,0,0
3,459909,420570,454865,447849,385822,323878,334512,331492,257727,156821,...,0,0,0,0,0,0,0,0,0,0
4,55264,43832,38151,39486,30967,32077,33663,33256,30886,28034,...,0,0,0,0,0,0,0,0,0,0


In [141]:
# Split our preprocessed data into our features and target arrays
y = energy_cancer_wdummies.LungBronchus_Classifier.values
X = energy_cancer_wdummies.drop(columns="LungBronchus_Classifier").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [142]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [146]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation='relu', input_dim=239))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 400)               96000     
                                                                 
 dense_13 (Dense)            (None, 50)                20050     
                                                                 
 dense_14 (Dense)            (None, 1)                 51        
                                                                 
Total params: 116101 (453.52 KB)
Trainable params: 116101 (453.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [147]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [150]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
2/2 [==============================] - 0s 120ms/step - loss: 8.9607e-04 - accuracy: 1.0000 - val_loss: 1918616.2500 - val_accuracy: 0.0769
Epoch 2/10
2/2 [==============================] - 0s 96ms/step - loss: 8.6638e-04 - accuracy: 1.0000 - val_loss: 1932174.1250 - val_accuracy: 0.0769
Epoch 3/10
2/2 [==============================] - 0s 95ms/step - loss: 8.4351e-04 - accuracy: 1.0000 - val_loss: 1946577.5000 - val_accuracy: 0.0769
Epoch 4/10
2/2 [==============================] - 0s 66ms/step - loss: 8.1642e-04 - accuracy: 1.0000 - val_loss: 1961341.2500 - val_accuracy: 0.0769
Epoch 5/10
2/2 [==============================] - 0s 76ms/step - loss: 7.9176e-04 - accuracy: 1.0000 - val_loss: 1973935.7500 - val_accuracy: 0.0769
Epoch 6/10
2/2 [==============================] - 0s 80ms/step - loss: 7.7057e-04 - accuracy: 1.0000 - val_loss: 1984477.1250 - val_accuracy: 0.0769
Epoch 7/10
2/2 [==============================] - 0s 51ms/step - loss: 7.4748e-04 - accuracy: 1.0000 - va

In [151]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.5833 - accuracy: 0.6154 - 24ms/epoch - 24ms/step
Loss: 0.5833035111427307, Accuracy: 0.6153846383094788


In [174]:
# Model test 2
import requests
url = "https://raw.githubusercontent.com/fraserclaire/Group3_Project4/main/Artien/Machine%20Learning%20Dataset-Cancer%26Energy.csv"
response = requests.get(url)
# with open("Machine Learning Dataset-Cancer&Energy.csv", "wb") as f:
#     f.write(response.content)

spark.sparkContext.addFile(url)
energy_cancer = spark.read.csv(SparkFiles.get('Machine Learning Dataset-Cancer&Energy.csv'),inferSchema=True,header=True)

# Convert Spark DataFrame to pandas DataFrame
energy_cancer_df = energy_cancer.toPandas()

energy_cancer_df.drop(columns=['_c0', 'State Abbreviation', 'State'], inplace=True)
energy_cancer_df.rename(columns={'Lung/Bronchus_Classifer': 'LungBronchus_Classifier'}, inplace=True)
energy_cancer_df.head()

,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),2020(C),...,2019_Population,2020_LungBronchus,2020_LungBronchus_perCapita,2020_Population,2021 Lung & Bronchus,2021_Population,2021_LungBronchus_perCapita,2011-2020 allsites/capita AVG,2011-2020 lungBronchus/capita AVG,LungBronchus_Classifier
0,15481,15521,14819,18225,19511,16613,16376,17325,17620,18527,...,731545,400,0.000546,732964.0,370,734923,0.000503,0.004628,0.000591,0
1,651032,547004,565051,575912,494311,410168,378890,377152,317180,256687,...,4903185,4230,0.000841,5031864.0,4520,5050380,0.000895,0.005554,0.000867,1
2,306119,296732,327099,339214,226889,246437,267591,304145,239798,161980,...,3017804,2760,0.000916,3014348.0,2970,3028443,0.000981,0.005473,0.000898,1
3,459909,420570,454865,447849,385822,323878,334512,331492,257727,156821,...,7278717,4200,0.000584,7186683.0,4550,7272487,0.000626,0.004939,0.000595,1
4,55264,43832,38151,39486,30967,32077,33663,33256,30886,28034,...,39512223,18040,0.000457,39503200.0,17760,39145060,0.000454,0.004456,0.000473,0


In [175]:
# Split our preprocessed data into our features and target arrays
y = energy_cancer_df.LungBronchus_Classifier.values
X = energy_cancer_df.drop(columns="LungBronchus_Classifier").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [176]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [180]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=250, activation='relu', input_dim=135))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 250)               34000     
                                                                 
 dense_28 (Dense)            (None, 50)                12550     
                                                                 
 dense_29 (Dense)            (None, 1)                 51        
                                                                 
Total params: 46601 (182.04 KB)
Trainable params: 46601 (182.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [181]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [182]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
2/2 [==============================] - 1s 178ms/step - loss: 0.8754 - accuracy: 0.4359 - val_loss: 1533008.2500 - val_accuracy: 0.0769
Epoch 2/10
2/2 [==============================] - 0s 30ms/step - loss: 0.6417 - accuracy: 0.7692 - val_loss: 1066656.2500 - val_accuracy: 0.0769
Epoch 3/10
2/2 [==============================] - 0s 29ms/step - loss: 0.5260 - accuracy: 0.7949 - val_loss: 777867.2500 - val_accuracy: 0.0769
Epoch 4/10
2/2 [==============================] - 0s 32ms/step - loss: 0.4855 - accuracy: 0.7949 - val_loss: 744029.0625 - val_accuracy: 0.0769
Epoch 5/10
2/2 [==============================] - 0s 29ms/step - loss: 0.4538 - accuracy: 0.8205 - val_loss: 792876.9375 - val_accuracy: 0.0769
Epoch 6/10
2/2 [==============================] - 0s 29ms/step - loss: 0.4367 - accuracy: 0.8205 - val_loss: 876537.6875 - val_accuracy: 0.0769
Epoch 7/10
2/2 [==============================] - 0s 31ms/step - loss: 0.4236 - accuracy: 0.7949 - val_loss: 995687.5625 - val_accura

In [183]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.5219 - accuracy: 0.6154 - 21ms/epoch - 21ms/step
Loss: 0.5218828916549683, Accuracy: 0.6153846383094788


In [186]:
## Model 3
# Remake dataframe, eliminating specific
model3 = energy_cancer_df[['2011(C)', '2012(C)', '2013(C)', '2014(C)', '2015(C)', '2016(C)',
                           '2017(C)', '2018(C)', '2019(C)', '2020(C)','2011(TR)', '2012(TR)',
                           '2013(TR)', '2014(TR)', '2015(TR)', '2016(TR)', '2017(TR)',
                           '2018(TR)', '2019(TR)', '2020(TR)', 'LungBronchus_Classifier']]

model3.head()

,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),2020(C),...,2012(TR),2013(TR),2014(TR),2015(TR),2016(TR),2017(TR),2018(TR),2019(TR),2020(TR),LungBronchus_Classifier
0,15481,15521,14819,18225,19511,16613,16376,17325,17620,18527,...,17831,18931,22760,23819,26254,24447,24996,23608,25527,0
1,651032,547004,565051,575912,494311,410168,378890,377152,317180,256687,...,265964,336863,293890,284365,259513,285081,301170,298545,308209,1
2,306119,296732,327099,339214,226889,246437,267591,304145,239798,161980,...,125377,132370,131625,129264,127831,121635,124834,132189,118123,1
3,459909,420570,454865,447849,385822,323878,334512,331492,257727,156821,...,114315,119753,139836,149031,162028,172736,179533,174570,177618,1
4,55264,43832,38151,39486,30967,32077,33663,33256,30886,28034,...,818490,897358,903692,911138,1116287,1314777,1213958,1373488,1244897,0


In [187]:
# Split our preprocessed data into our features and target arrays
y = model3.LungBronchus_Classifier.values
X = model3.drop(columns="LungBronchus_Classifier").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [188]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [189]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu', input_dim=20))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 50)                1050      
                                                                 
 dense_31 (Dense)            (None, 2)                 102       
                                                                 
 dense_32 (Dense)            (None, 1)                 3         
                                                                 
Total params: 1155 (4.51 KB)
Trainable params: 1155 (4.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [190]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [193]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
2/2 [==============================] - 0s 73ms/step - loss: 0.6419 - accuracy: 0.6923 - val_loss: 14096.3398 - val_accuracy: 0.4615
Epoch 2/50
2/2 [==============================] - 0s 49ms/step - loss: 0.6414 - accuracy: 0.6923 - val_loss: 14579.3281 - val_accuracy: 0.4615
Epoch 3/50
2/2 [==============================] - 0s 28ms/step - loss: 0.6408 - accuracy: 0.6923 - val_loss: 15162.1406 - val_accuracy: 0.4615
Epoch 4/50
2/2 [==============================] - 0s 29ms/step - loss: 0.6402 - accuracy: 0.6923 - val_loss: 15784.8369 - val_accuracy: 0.4615
Epoch 5/50
2/2 [==============================] - 0s 31ms/step - loss: 0.6396 - accuracy: 0.6923 - val_loss: 16370.8086 - val_accuracy: 0.4615
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 0.6392 - accuracy: 0.6923 - val_loss: 15999.8643 - val_accuracy: 0.4615
Epoch 7/50
2/2 [==============================] - 0s 36ms/step - loss: 0.6389 - accuracy: 0.6923 - val_loss: 15801.1416 - val_accuracy: 0.4615

In [192]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.4966 - accuracy: 0.9231 - 126ms/epoch - 126ms/step
Loss: 0.49664244055747986, Accuracy: 0.9230769276618958


In [194]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1/1 - 0s - loss: 0.4470 - accuracy: 0.9231 - 33ms/epoch - 33ms/step
Loss: 0.44697701930999756, Accuracy: 0.9230769276618958
